# Task 01 - Teoría

### Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models

### Investigue qué son los factorial HMM (Hidden Markov Models)

### Especifique en sus propias palabras el algoritmo Forward Backward para HMM

### En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)

# Task 02 - Algoritmo Forward Backward en HMM

## Parámetros iniciales

In [1]:
# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Sunny']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}

## Clase HMM

In [2]:
class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob
        
    def generate_sequence(self, length):
        import random
        sequence = []
        state = random.choice(self.states)
        sequence.append(state)
        for i in range(length-1):
            next_state = random.choices(self.states, weights=[self.transition_prob[state][s] for s in self.states])[0]
            sequence.append(next_state)
            state = next_state
        return sequence
    
    def forward(self, observations):
        alpha = []
        for i in range(len(self.states)):
            alpha.append([0]*len(observations))
        for i in range(len(self.states)):
            alpha[i][0] = self.initial_prob[self.states[i]] * self.emission_prob[self.states[i]][observations[0]]
        for t in range(1, len(observations)):
            for i in range(len(self.states)):
                alpha[i][t] = self.emission_prob[self.states[i]][observations[t]] * sum(alpha[j][t-1] * self.transition_prob[self.states[j]][self.states[i]] for j in range(len(self.states)))
        return alpha
    
    def backward(self, observations):
        beta = []
        for i in range(len(self.states)):
            beta.append([0]*len(observations))
        for i in range(len(self.states)):
            beta[i][-1] = 1
        for t in range(len(observations)-2, -1, -1):
            for i in range(len(self.states)):
                beta[i][t] = sum(self.transition_prob[self.states[i]][self.states[j]] * self.emission_prob[self.states[j]][observations[t+1]] * beta[j][t+1] for j in range(len(self.states)))
        return beta
    
    def compute_state_probabilities(self, observations):
        alpha = self.forward(observations)
        beta = self.backward(observations)
        prob = []
        for t in range(len(observations)):
            prob_t = [0]*len(self.states)
            for i in range(len(self.states)):
                prob_t[i] = alpha[i][t] * beta[i][t]
            prob.append(prob_t)
        # Normalizar las probabilidades
        prob = [[p/sum(p_t) for p in p_t] for p_t in prob]
        return prob

## Instancia de la clase HMM

In [3]:
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

## Secuencia de Observaciones

In [4]:
obs_sequence = hmm.generate_sequence(5)
print("Secuencia generada: ", obs_sequence)

Secuencia generada:  ['Sunny', 'Rainy', 'Rainy', 'Sunny', 'Rainy']


## Cálculo de probabilidades forward

In [5]:
forward_prob = hmm.forward(observations)
print("Probabilidades forward: ", forward_prob)

Probabilidades forward:  [[0.4, 0.30400000000000005, 0.21088000000000007], [0.15, 0.051000000000000004, 0.027420000000000003]]


## Cálculo de probabilidades backward

In [6]:
backward_prob = hmm.backward(observations)
print("Probabilidades backward: ", backward_prob)

Probabilidades backward:  [[0.4780000000000002, 0.7000000000000002, 1], [0.31400000000000006, 0.5, 1]]


## Calcular probabilidades del estado

In [7]:
state_prob = hmm.compute_state_probabilities(observations)
print("Probabilidades de estados: ", state_prob)

Probabilidades de estados:  [[0.8023499790180445, 0.19765002098195547], [0.8929920268569032, 0.1070079731430969], [0.8849349559378934, 0.11506504406210658]]
